In [1]:
import causal_nf.utils.wandb_local as wandb_local
import causal_nf.config as causal_nf_config
from causal_nf.config import cfg
import causal_nf.utils.training as causal_nf_train
import os
import glob
import argparse
from yacs.config import CfgNode 
import torch
import causal_nf.utils.io as causal_nf_io
import time
import numpy as np
import os

/Users/llaurabat/micromamba/envs/causal-flows/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from causal_nf.preparators.FF_preparator import FFPreparator
from causal_nf.config import cfg


In [20]:
info_dict = {'M1_ckpt_files':["/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_M1_1/a6cbbd08526111efa71facde48001122/last.ckpt",
                             "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_M1_2/a6cbbd08526111efa71facde48001122/last.ckpt",
                             "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_M1_3/a6cbbd08526111efa71facde48001122/last.ckpt",
                             "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_M1_4/a6cbbd08526111efa71facde48001122/last.ckpt",
                             "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_M1_5/a6cbbd08526111efa71facde48001122/last.ckpt"],
            'M2_ckpt_files':["/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_M2_1/a6cbbd08526111efa71facde48001122/last.ckpt",
                            "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_M2_2/a6cbbd08526111efa71facde48001122/last.ckpt",
                            "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_M2_3/a6cbbd08526111efa71facde48001122/last.ckpt",
                            "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_M2_4/a6cbbd08526111efa71facde48001122/last.ckpt",
                            "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1_M2_5/a6cbbd08526111efa71facde48001122/last.ckpt"],}


In [ ]:
results = {}

In [23]:
for mod_ix, (mod, check_files) in enumerate(info_dict.items()):
    results[f'M{mod_ix+1}'] = []
    for c_ix, check_file in enumerate(check_files):
        components = os.path.normpath(check_file).split(os.sep)
        folder = components[-3]
        check_code = components[-2]

        args_list = []
        args =  CfgNode({'config_file': f'{folder}/{check_code}/wandb_local/config_local.yaml', 
                         'config_default_file': f'{folder}/{check_code}/wandb_local/default_config.yaml', 
                         'project': None, 'wandb_mode': 'disabled', 'wandb_group': None, 
                         'load_model': f'{folder}/{check_code}', 'delete_ckpt': False})
        config = causal_nf_config.build_config(
        config_file=args.config_file,
            args_list=args_list,
            config_default_file=args.config_default_file,
        )
        causal_nf_config.assert_cfg_and_config(cfg, config)
        preparator = FFPreparator.loader(cfg.dataset)
        preparator.prepare_data()
        model_lightning = causal_nf_train.load_model(cfg=cfg, preparator=preparator, ckpt_file=check_file)
        model = model_lightning.model
        model.eval()
        loaders = preparator.get_dataloaders(
            batch_size=cfg.train.batch_size, num_workers=cfg.train.num_workers
        )
        
        # for i, loader in enumerate(loaders):
        #     causal_nf_io.print_info(f"[{i}] num_batchees: {len(loader)}")

        # manual implementation of my_predict
        n_rounds = 5
        ates = []
        seeds = np.arange(n_rounds)
        batch = next(iter(loaders[-1]))
        for i, seed in enumerate(seeds):
            output = {}
            x = batch[0] # is this u or x? I hope x
            n = x.shape[0]
            with torch.enable_grad():
                output["log_prob_true"] = preparator.log_prob(x)
            
            log_prob = model.log_prob(x, scaler=preparator.scaler_transform)
            output["loss"] = -log_prob
            output["log_prob"] = log_prob
            # intervention_list = preparator.get_ate_list()
            
            int_dict = {'name': '1_0', 'a': 1., 'b': 0., 'index': 4}
            
        
            name = int_dict["name"]
            a = int_dict["a"]#1.
            b = int_dict["b"]#0.
            index = int_dict["index"]
        
            torch.random.manual_seed(seed) 
            ate = model_lightning.model.compute_ate(
                index,
                a=a,
                b=b,
                num_samples=10000,
                scaler=preparator.scaler_transform,
            )
            
            ates.append(ate.detach().numpy())
        
        ates = np.array(ates)
        ates_y = ates[:,-1]
        mean = ates_y.mean()
        std =  ates_y.std()
        print(f'Model {mod_ix+1} dataset {c_ix+1} mean:', mean)
        print(f'Model {mod_ix+1} dataset {c_ix+1} std:', std)
        results[f'M{mod_ix+1}'].append(ates_y)
        

        



Preparing data...

Preparing data...

Preparing data...
NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, 5]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=6, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=138, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFightning'> from /Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1/a6cbbd08526111efa71facde48001122/last.ckpt
scaler_transform StandardTransform(shift=tensor([2.7234, 2.7408, 2.7082, 2.6903, 0.9523, 1.9587]), scale=tensor([

In [24]:
# ckpt_file = "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final/edf3624c525411ef8eecacde48001122/last.ckpt"
# ckpt_file = "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_adj_ON/1f995ea8525c11efbcefacde48001122/last.ckpt"
ckpt_file = "/Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1/a6cbbd08526111efa71facde48001122/last.ckpt"

In [25]:
args_list = []
args =  CfgNode({'config_file': 'output_FF_final_ate1/a6cbbd08526111efa71facde48001122/wandb_local/config_local.yaml', 
                 'config_default_file': 'output_FF_final_ate1/a6cbbd08526111efa71facde48001122/wandb_local/default_config.yaml', 
                 'project': None, 'wandb_mode': 'disabled', 'wandb_group': None, 
                 'load_model': 'output_FF_final_ate1/a6cbbd08526111efa71facde48001122', 'delete_ckpt': False})

In [26]:
# args_list, args = causal_nf_config.parse_args()

In [27]:
config = causal_nf_config.build_config(
    config_file=args.config_file,
    args_list=args_list,
    config_default_file=args.config_default_file,
)

In [28]:
cfg.dataset

CfgNode({'root': '../Data', 'name': 'FF_data', 'sem_name': 'dummy', 'splits': [0.8, 0.1, 0.1], 'k_fold': 1, 'shuffle_train': True, 'single_split': False, 'loss': 'default', 'scale': 'default', 'num_samples': 1000, 'base_version': 1, 'add_noise': False, 'type': 'torch', 'use_edge_attr': False})

In [29]:
causal_nf_config.assert_cfg_and_config(cfg, config)

In [30]:
preparator = FFPreparator.loader(cfg.dataset)

In [31]:
preparator.prepare_data()


Preparing data...

Preparing data...

Preparing data...


In [32]:
# IMPORTANT: insert ckpt file otherwise it loads anyway bt something wrong
model_lightning = causal_nf_train.load_model(cfg=cfg, preparator=preparator, ckpt_file=ckpt_file)

NSF(
  (transforms): ModuleList(
    (0): MaskedAutoregressiveTransform(
      (base): MonotonicRQSTransform(bins=8)
      (order): [0, 1, 2, 3, 4, 5]
      (hyper): MaskedMLP(
        (0): MaskedLinear(in_features=6, out_features=32, bias=True)
        (1): ELU(alpha=1.0)
        (2): MaskedLinear(in_features=32, out_features=32, bias=True)
        (3): ELU(alpha=1.0)
        (4): MaskedLinear(in_features=32, out_features=32, bias=True)
        (5): ELU(alpha=1.0)
        (6): MaskedLinear(in_features=32, out_features=138, bias=True)
      )
    )
  )
  (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
)
[INFO] Loading <class 'causal_nf.models.causal_nf.CausalNFightning'> from /Users/llaurabat/Dropbox/Oxford/deep_copula_frugal/causal-flows/output_FF_final_ate1/a6cbbd08526111efa71facde48001122/last.ckpt
scaler_transform StandardTransform(shift=tensor([2.7234, 2.7408, 2.7082, 2.6903, 0.9523, 1.9587]), scale=tensor([2.7295, 2.6963, 2.7701, 2.7185, 0.2133, 1.0559]))
Standar

In [33]:
# list(model.model.named_parameters())

In [34]:
model = model_lightning.model
model.eval()

CausalNormalizingFlow(
  (flow): NSF(
    (transforms): ModuleList(
      (0): MaskedAutoregressiveTransform(
        (base): MonotonicRQSTransform(bins=8)
        (order): [0, 1, 2, 3, 4, 5]
        (hyper): MaskedMLP(
          (0): MaskedLinear(in_features=6, out_features=32, bias=True)
          (1): ELU(alpha=1.0)
          (2): MaskedLinear(in_features=32, out_features=32, bias=True)
          (3): ELU(alpha=1.0)
          (4): MaskedLinear(in_features=32, out_features=32, bias=True)
          (5): ELU(alpha=1.0)
          (6): MaskedLinear(in_features=32, out_features=138, bias=True)
        )
      )
    )
    (base): DiagNormal(loc: torch.Size([6]), scale: torch.Size([6]))
  )
)

In [35]:
loaders = preparator.get_dataloaders(
    batch_size=cfg.train.batch_size, num_workers=cfg.train.num_workers
)

for i, loader in enumerate(loaders):
    causal_nf_io.print_info(f"[{i}] num_batchees: {len(loader)}")


[INFO] [0] num_batchees: 1
[INFO] [1] num_batchees: 1
[INFO] [2] num_batchees: 1


In [36]:
batch = next(iter(loaders[-1]))

In [37]:
batch[0].shape

torch.Size([1000, 6])

In [38]:
# predict from model class
# torch.random.manual_seed(8)
# loss_dict = model_lightning.predict(
#     batch=next(iter(loaders[2])),
#     observational=False,
#     intervene=False,
#     counterfactual=False,
#     ate=True,
# )


In [39]:
# manual implementation of my_predict
n_rounds = 25
ates = []
seeds = np.arange(n_rounds)
for i, seed in enumerate(seeds):
    output = {}
    x = batch[0] # is this u or x? I hope x
    n = x.shape[0]
    with torch.enable_grad():
        output["log_prob_true"] = preparator.log_prob(x)
    
    log_prob = model.log_prob(x, scaler=preparator.scaler_transform)
    output["loss"] = -log_prob
    output["log_prob"] = log_prob
    # intervention_list = preparator.get_ate_list()
    
    int_dict = {'name': '1_0', 'a': 1., 'b': 0., 'index': 4}
    

    name = int_dict["name"]
    a = int_dict["a"]#1.
    b = int_dict["b"]#0.
    index = int_dict["index"]

    torch.random.manual_seed(seed) 
    ate = model_lightning.model.compute_ate(
        index,
        a=a,
        b=b,
        num_samples=10000,
        scaler=preparator.scaler_transform,
    )
    
    ates.append(ate.detach().numpy())

ates = np.array(ates)
ates_y = ates[:,-1]
mean = ates_y.mean()
std =  ates_y.std()
print('mean:', mean)
print('std:', std)

mean: 0.7469618
std: 0.013235588


In [65]:
def my_predict(
        self,
        batch,
        observational=False,
        intervene=False,
        counterfactual=False,
        ate=False,
        intervention_list=None,
    ):
        output = {}
        x = batch[0].to(self.device)
        n = x.shape[0]
        with torch.enable_grad():
            output["log_prob_true"] = self.preparator.log_prob(x)

        tic = time.time()
        log_prob = self.model.log_prob(x, scaler=self.preparator.scaler_transform)
        output["time_log_prob"] = self.compute_time(tic, n)
        output["loss"] = -log_prob
        output["log_prob"] = log_prob

        if ate:
            if intervention_list is None:
                intervention_list = self.preparator.get_ate_list()
            delta_times = []
            for int_dict in intervention_list:
                name = int_dict["name"]
                a = int_dict["a"]
                b = int_dict["b"]
                index = int_dict["index"]
                tic = time.time()
                ate = self.model.compute_ate(
                    index,
                    a=a,
                    b=b,
                    num_samples=10000,
                    scaler=self.preparator.scaler_transform,
                )
                delta_times.append(self.compute_time(tic, 10000))

                ate_true = self.preparator.compute_ate(
                    index, a=a, b=b, num_samples=10000
                )

                diff_ate = ate_true - ate

                rmse = torch.sqrt((diff_ate**2).sum())
                output[f"rmse_ate_x{index + 1}={name}"] = rmse

            delta_time = torch.stack(delta_times).mean()
            output["time_ate"] = delta_time

        return output

In [66]:
intervention_list = [{'name': '1_0', 'a': 1., 'b': 0., 'index': 0},]

In [67]:
bri = my_predict(
        self=model_lightning,
        batch=batch,
        observational=False,
        intervene=False,
        counterfactual=False,
        ate=True,
        intervention_list=intervention_list,
    )
bri

{'log_prob_true': tensor([-2.8672, -2.4891, -6.4312,  ..., -4.2171, -2.1910, -1.8383]),
 'time_log_prob': tensor(0.9432),
 'loss': tensor([2.5407, 2.3248, 7.3806,  ..., 4.2023, 2.1255, 2.3977],
        grad_fn=<NegBackward0>),
 'log_prob': tensor([-2.5407, -2.3248, -7.3806,  ..., -4.2023, -2.1255, -2.3977],
        grad_fn=<AddBackward0>),
 'rmse_ate_x1=1_0': tensor(0.0135),
 'time_ate': tensor(4.4213)}

In [ ]:
python main.py --config_file causal_nf/configs/FF_configs.yaml --wandb_mode disabled --project FF_data
